In [149]:
import requests
import json
#import aiohttp
#import asyncio
from io import StringIO
from html.parser import HTMLParser
import pandas as pd
import time
from tqdm import tqdm

In [150]:
PATH = "files"

In [151]:
#claning text from html tags 

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = StringIO()

    def handle_data(self, d):
        self.text.write(d)

    def get_data(self):
        return self.text.getvalue()


def strip_tags(html):
    s = MLStripper()
    if not html:
        return ""
    s.feed(html)
    return s.get_data()

In [213]:
class Parser:
    def __init__(self, text, area, per_page):
        self.text = "NAME: " + text
        self.area = area
        self.per_page = per_page

    def getPage(self, p=0):
        params = {
            'text': self.text,  # Текст фильтра. В имени должно быть слово "Аналитик"
            'area': self.area,  # Поиск ощуществляется по вакансиям города Москва
            'page': p,  # Индекс страницы поиска на HH
            'per_page': self.per_page  # Кол-во вакансий на 1 странице
        }

        req = requests.get('https://api.hh.ru/vacancies', params)  # Посылаем запрос к API
        data = req.content.decode()  # Декодируем его ответ, чтобы Кириллица отображалась корректно
        req.close()
        return data
    
    
    def process_key_skills(self, skills):
        return ", ".join([skill["name"] for skill in skills])

    def make_save_df(self, pages, name):
        df = self.make_df(pages)
        df.to_csv(name, index=False)
        
    def getPages(self, n_pages):
        urls = []
        snippets = []
        for i in range(n_pages):
            data = json.loads((self.getPage(i)))
            
            
            urls = urls + [elem["url"] for elem in data["items"]]
            snippets = snippets + [elem["snippet"] for elem in data["items"]]
        return urls, snippets

    def Parse(self, pages):
        urls, snippets = self.getPages(pages)
        # loop = asyncio.get_event_loop()
        # htmls = loop.run_until_complete(self.get(urls, loop))
        # loop.close()
        htmls = []
        for url in urls:
            req = requests.get(url)
            html = req.content.decode()
            req.close()
            htmls.append(html)
        htmls = [json.loads(elem) for elem in htmls]
        return htmls, snippets
    
    def make_df(self, pages):
        df = pd.DataFrame(columns=['id', 'name', 'skills', 'desc', 'experience', 'spec', "area","resp","req",
                                   "schedule","employment", "profarea_name","url",
                                   "address", "lat", "lng"],
                          index=range(pages * self.per_page))
        
        rows, snippets = self.Parse(pages)
        for i, row in enumerate(rows):
            df.iloc[i]["id"] = row["id"]
            df.iloc[i]["name"] = row["name"]
            df.iloc[i]["skills"] = self.process_key_skills(row["key_skills"])
            df.iloc[i]["desc"] = strip_tags(row["description"])
            df.iloc[i]["experience"] = row["experience"]["name"]
            df.iloc[i]["spec"] = self.process_key_skills(row["specializations"])
            df.iloc[i]["area"] = row["area"]["name"]
            df.iloc[i]["resp"] = strip_tags(snippets[i]["responsibility"])
            df.iloc[i]["req"] = strip_tags(snippets[i]["requirement"])
            df.iloc[i]["schedule"] = row["schedule"]['name']
            df.iloc[i]["employment"] = row["employment"]['name']
            df.iloc[i]["profarea_name"] = row["specializations"][0]["profarea_name"]
            df.iloc[i]["url"] = row["alternate_url"]
            df.iloc[i]["address"] = None if type(row["address"]) == type(None) else row["address"]["raw"]
            df.iloc[i]["lat"] = None if type(row["address"]) == type(None) else row["address"]["lat"]
            df.iloc[i]["lng"] = None if type(row["address"]) == type(None) else row["address"]["lng"]
            

        return df


In [214]:
m1 = Parser('Аналитик', 1, 3)

In [215]:
m1.make_df(1)

,id,name,skills,desc,experience,spec,area,resp,req,schedule,employment,profarea_name,url,address,lat,lng
0,44490685,Младший аналитик данных,"Python, Django Framework, Английский язык, Dat...",Обязанности: Решать практические NLP-задачи д...,От 1 года до 3 лет,"Начальный уровень, Мало опыта",Москва,Решать практические NLP-задачи для русского яз...,"Знание основ NLP (named entity recognition, in...",Полный день,Полная занятость,"Информационные технологии, интернет, телеком",https://hh.ru/vacancy/44490685,None,None,None
1,46424733,Corporate Development –Analyst / Junior Analyst,"Corporate Finance, Business Analysis, Analytic...",Zigmund.Online is a leading fast-growing compa...,От 1 года до 3 лет,"Бюджетирование и планирование, Планово-экономи...",Москва,Preparation and updating of the financial mode...,Minimum 1 or 2 years of similar experience in ...,Полный день,Полная занятость,"Бухгалтерия, управленческий учет, финансы пред...",https://hh.ru/vacancy/46424733,None,None,None
2,44940321,Junior Product manager,"Маркетинговый анализ, Сбор и анализ информации...","Привет! Если ты ждал знак, то вот он :) Гигант...",От 1 года до 3 лет,"Начальный уровень, Мало опыта, Менеджмент прод...",Москва,Для бизнеса - платформа для поиска линейных ис...,"Здравый смысл и желание работать головой, а не...",Полный день,Полная занятость,"Маркетинг, реклама, PR",https://hh.ru/vacancy/44940321,"Москва, улица Ленинская Слобода, 19",55.7083,37.6528


In [254]:
class Df_Maker:
    def __init__(self, params):
        self.params = params
       
    def create_df(self):
        data = pd.DataFrame()
        for value in params:
            parsing = Parser(value['text'], value['area'], value['per_page'])
            sub_df = parsing.make_df(1)
            data = data.append(sub_df, ignore_index = True)
        return data


In [255]:
params = [{'text': 'Аналитик', 'area': 1, 'per_page': 3},
          {'text': 'data science', 'area': 1, 'per_page': 2}]

In [256]:
c = Df_Maker(params)

In [257]:
c.params

[{'text': 'Аналитик', 'area': 1, 'per_page': 3},
 {'text': 'data science', 'area': 1, 'per_page': 2}]

In [258]:
c.create_df()

,id,name,skills,desc,experience,spec,area,resp,req,schedule,employment,profarea_name,url,address,lat,lng
0,44490685,Младший аналитик данных,"Python, Django Framework, Английский язык, Dat...",Обязанности: Решать практические NLP-задачи д...,От 1 года до 3 лет,"Начальный уровень, Мало опыта",Москва,Решать практические NLP-задачи для русского яз...,"Знание основ NLP (named entity recognition, in...",Полный день,Полная занятость,"Информационные технологии, интернет, телеком",https://hh.ru/vacancy/44490685,None,None,None
1,46424733,Corporate Development –Analyst / Junior Analyst,"Corporate Finance, Business Analysis, Analytic...",Zigmund.Online is a leading fast-growing compa...,От 1 года до 3 лет,"Бюджетирование и планирование, Планово-экономи...",Москва,Preparation and updating of the financial mode...,Minimum 1 or 2 years of similar experience in ...,Полный день,Полная занятость,"Бухгалтерия, управленческий учет, финансы пред...",https://hh.ru/vacancy/46424733,None,None,None
2,44940321,Junior Product manager,"Маркетинговый анализ, Сбор и анализ информации...","Привет! Если ты ждал знак, то вот он :) Гигант...",От 1 года до 3 лет,"Начальный уровень, Мало опыта, Менеджмент прод...",Москва,Для бизнеса - платформа для поиска линейных ис...,"Здравый смысл и желание работать головой, а не...",Полный день,Полная занятость,"Маркетинг, реклама, PR",https://hh.ru/vacancy/44940321,"Москва, улица Ленинская Слобода, 19",55.7083,37.6528
3,46663886,SQL Data Engineer/ CRM Analyst,"SQL, Data Mining, Data Transformation, ETL, Go...",To further evolve our product and scale the bu...,От 3 до 6 лет,"Программирование, Разработка, Инженер, Стартапы",Москва,Constantly innovate on our overall data archit...,"A degree in Computer Science, Math, Statistics...",Удаленная работа,Полная занятость,"Информационные технологии, интернет, телеком",https://hh.ru/vacancy/46663886,None,None,None
4,45992609,Senior Data Scientist,"Tensorflow, Python, Machine Learning, Data Sci...",We are hiring an ambitious data scientist to t...,От 3 до 6 лет,"Программирование, Разработка, Аналитик",Москва,Designing and documenting data architecture at...,5+ years of professional experience as a data ...,Удаленная работа,Полная занятость,"Информационные технологии, интернет, телеком",https://hh.ru/vacancy/45992609,None,None,None
